In [77]:
import pandas as pd
import os
import json

from organizeEventData import *

In [78]:
country = 'spain'

In [79]:
matchesFile = open(f"data/matches/matches_{country}.json")
matches = json.load(matchesFile)
matchesMap = {}
for match in matches:
    matchesMap[match["wyId"]] = match
matchesFile.close()

teamsFile = open("data/teams.json")
teams = json.load(teamsFile)
teamsFile.close()

playersFile = open("data/players.json")
players = json.load(playersFile)
playerMap = {}
for player in players:
    playerMap[player["wyId"]] =player
playersFile.close()

eventsFile = open(f"data/events/events_{country}.json")
events = json.load(eventsFile)
eventsFile.close()

#going through each match, splitting up the events by event type and then calculating the number of each event in that match
eventsPerMatch = groupEventsByMatch(events)
for match in eventsPerMatch:
    teamIds = getTeamIdsFromMatchId(match,matches,teams)
    team1Name = teamNameFromId(int(teamIds[0]), teams)
    team2Name = teamNameFromId(int(teamIds[1]), teams)
    # print(team1Name + " vs " + team2Name)
    eventsPerMatch[match] = splitEventsByType(eventsPerMatch[match])
    # numDicts = calcNumByEventType(eventsPerMatch[match],teamIds)
    # print(team1Name)
    # for event in numDicts[0]:
    #     print("Number of " + event + ": " + str(numDicts[0][event]))
    # print(team2Name)
    # for event in numDicts[1]:
    #     print("Number of " + event + ": " + str(numDicts[1][event]))
    # print("---------------")


Done 10 matches
Done 20 matches
Done 30 matches
Done 40 matches
Done 50 matches
Done 60 matches
Done 70 matches
Done 80 matches
Done 90 matches
Done 100 matches
Done 110 matches
Done 120 matches
Done 130 matches
Done 140 matches
Done 150 matches
Done 160 matches
Done 170 matches
Done 180 matches
Done 190 matches
Done 200 matches
Done 210 matches
Done 220 matches
Done 230 matches
Done 240 matches
Done 250 matches
Done 260 matches
Done 270 matches
Done 280 matches
Done 290 matches
Done 300 matches
Done 310 matches
Done 320 matches
Done 330 matches
Done 340 matches
Done 350 matches
Done 360 matches
Done 370 matches
Done 380 matches


In [80]:
df_matches = pd.read_json(f"/Users/tw/Desktop/Deep_Learning/Project/data/matches/matches_{country}.json")[['wyId', 'label', 'teamsData', 'winner']]

In [81]:
def clean_team_names(team):
    return team_dict[team]

def extract_label_information(label):
    teams, score = label.split(',')
    home_team, away_team = [x for x in teams.split(' - ')]
    home_goals, away_goals = [int(x.replace(' ', '')) for x in score.split(' - ')]

    return home_team, away_team, home_goals, away_goals

def get_team_ids(teamsData):
    return list(teamsData.keys())

def extract_game_information(teamsData, teams):
    teamIds = list(teamsData.keys())
    
    for t_id in teamIds:
        if teamsData[t_id]['side'] == 'home':
            team1_id = t_id
            team1_name = teamNameFromId(int(t_id), teams)
            team1_goals = teamsData[t_id]['score']
        elif teamsData[t_id]['side'] == 'away': 
            team2_id = t_id
            team2_name = teamNameFromId(int(t_id), teams)
            team2_goals = teamsData[t_id]['score']
        else:
            print('ERROR: team ids')
    return team1_id, team1_name, team2_id, team2_name, team1_goals, team2_goals

df_matches['team1_id'], df_matches['team1_name'], df_matches['team2_id'], df_matches['team2_name'], df_matches['team1_goals'], df_matches['team2_goals'] = zip(*df_matches.apply(lambda x: extract_game_information(x['teamsData'], teams), axis='columns'))
# df_matches['team1_id'], df_matches['team2_id'] = zip(*df_matches.apply(lambda x: get_team_ids(x['teamsData']), axis='columns'))

In [82]:
df_matches['team1_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team1_id']), axis='columns')
df_matches['team2_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team2_id']), axis='columns')

In [83]:
df_matches = df_matches.drop(columns=['label', 'teamsData', 'winner'])
df_matches = df_matches.rename(columns={"wyId": "game_id"})

In [84]:
df_matches.sort_values(by='game_id')

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win
371,2565548,695,Levante,682,Villarreal,1,0,True,False
377,2565549,692,Celta de Vigo,687,Real Sociedad,2,3,False,True
374,2565550,678,Athletic Club,698,Getafe,0,0,False,False
376,2565551,756,Girona,679,Atl\u00e9tico Madrid,2,2,False,False
375,2565552,680,Sevilla,691,Espanyol,1,1,False,False
...,...,...,...,...,...,...,...,...,...
7,2565923,714,Las Palmas,756,Girona,1,2,False,True
3,2565924,674,Valencia,677,Deportivo La Coru\u00f1a,2,1,True,False
1,2565925,679,Atl\u00e9tico Madrid,701,Eibar,2,2,False,False
8,2565926,712,Legan\u00e9s,684,Real Betis,3,2,True,False


In [85]:
game_events = pd.DataFrame.from_records(eventsPerMatch).T
game_events = (game_events.reset_index()).rename(columns={"index": "game_id"})

In [86]:
df_matches = df_matches.merge(game_events, on='game_id')

In [87]:
#calculates the number of event type for a match (ie: number of corners) and return it:
def calcNumByEventType(events, team):
    count=0
    for bin in events:
        for event in events[bin]:
            if(event["teamId"]==int(team)):
                count+=1
    return count

for id in [1,2]:
    for event in ['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside']:
        df_matches[f'team{id}_{event}'] = df_matches.apply(lambda x: calcNumByEventType(x[event], x[f'team{id}_id']), axis='columns')


In [88]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,shot,...,team1_freeKickShots,team1_redCard,team1_yellowCard,team1_offside,team2_shot,team2_corner,team2_freeKickShots,team2_redCard,team2_yellowCard,team2_offside
371,2565548,695,Levante,682,Villarreal,1,0,True,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,0,0,1,2,7,7,0,0,3,1
377,2565549,692,Celta de Vigo,687,Real Sociedad,2,3,False,True,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,1,0,3,3,10,5,1,0,1,1
374,2565550,678,Athletic Club,698,Getafe,0,0,False,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,0,0,1,1,6,9,0,1,4,1
376,2565551,756,Girona,679,Atl\u00e9tico Madrid,2,2,False,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,1,0,2,0,8,3,0,1,4,2
375,2565552,680,Sevilla,691,Espanyol,1,1,False,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,0,1,2,5,9,6,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2565923,714,Las Palmas,756,Girona,1,2,False,True,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,1,0,1,1,12,6,0,0,2,3
3,2565924,674,Valencia,677,Deportivo La Coru\u00f1a,2,1,True,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,0,0,0,1,11,4,0,0,0,0
1,2565925,679,Atl\u00e9tico Madrid,701,Eibar,2,2,False,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,1,1,5,6,16,5,0,0,2,0
8,2565926,712,Legan\u00e9s,684,Real Betis,3,2,True,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,1,1,1,3,17,4,0,0,0,3


In [89]:
def getStartingLineupAverageHeight(match, playerMap):

    teamIds = list(match["teamsData"].keys())
    for t_id in teamIds:
        if match["teamsData"][t_id]['side'] == 'home':
            team1_id = t_id
        else:
            team2_id = t_id

    team1Height = 0
    team2Height = 0
    for team in teamIds:
        for player in match["teamsData"][team]["formation"]["lineup"]:
            playerId = player["playerId"]
            if (team == team1_id):
                team1Height+= playerMap[playerId]["height"]
            elif (team == team2_id):
                team2Height+= playerMap[playerId]["height"]

    return [team1_id, team1Height/11, team2_id, team2Height/11]

match_list = []
for matchId in matchesMap:
    match = [matchId] + getStartingLineupAverageHeight(matchesMap[matchId], playerMap)
    match_list.append(match)

In [90]:
df_height = pd.DataFrame(match_list, columns=['game_id', 'team1_id', 'team1_height', 'team2_id', 'team2_height'])

In [91]:
df_matches = df_matches.merge(df_height, on=['game_id', 'team1_id', 'team2_id'])

In [92]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,shot,...,team1_yellowCard,team1_offside,team2_shot,team2_corner,team2_freeKickShots,team2_redCard,team2_yellowCard,team2_offside,team1_height,team2_height
371,2565548,695,Levante,682,Villarreal,1,0,True,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,1,2,7,7,0,0,3,1,181.636364,180.727273
377,2565549,692,Celta de Vigo,687,Real Sociedad,2,3,False,True,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,3,3,10,5,1,0,1,1,179.545455,182.000000
374,2565550,678,Athletic Club,698,Getafe,0,0,False,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,1,1,6,9,0,1,4,1,182.454545,180.000000
376,2565551,756,Girona,679,Atl\u00e9tico Madrid,2,2,False,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,2,0,8,3,0,1,4,2,181.727273,182.545455
375,2565552,680,Sevilla,691,Espanyol,1,1,False,False,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,2,5,9,6,0,0,4,0,182.181818,179.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2565923,714,Las Palmas,756,Girona,1,2,False,True,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...",...,1,1,12,6,0,0,2,3,178.363636,182.181818
3,2565924,674,Valencia,677,Deportivo La Coru\u00f1a,2,1,True,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,0,1,11,4,0,0,0,0,181.909091,176.454545
1,2565925,679,Atl\u00e9tico Madrid,701,Eibar,2,2,False,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,5,6,16,5,0,0,2,0,182.363636,182.363636
8,2565926,712,Legan\u00e9s,684,Real Betis,3,2,True,False,"{'5': [], '10': [{'eventId': 10, 'subEventName...",...,1,3,17,4,0,0,0,3,181.909091,181.363636


In [93]:
# Reset DataFrame with columns in desired order
cols = ['game_id','team1_name','team1_id','team2_name','team2_id']
cols.extend(['team1_goals', 'team2_goals', 'team1_win', 'team2_win'])
cols.extend(['team1_height', 'team2_height'])


event_names = ['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside']

cols.extend([f'team1_{name}' for name in event_names])
cols.extend([f'team2_{name}' for name in event_names])

cols.extend(['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside'])

df_matches = df_matches[cols]

In [94]:
df_matches

,game_id,team1_name,team1_id,team2_name,team2_id,team1_goals,team2_goals,team1_win,team2_win,team1_height,...,team2_freeKickShots,team2_redCard,team2_yellowCard,team2_offside,shot,corner,freeKickShots,redCard,yellowCard,offside
0,2565922,Barcelona,676,Real Sociedad,687,1,0,True,False,181.636364,...,1,0,2,3,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...","{'5': [{'eventId': 3, 'subEventName': 'Corner'...","{'5': [], '10': [], '15': [], '20': [{'eventId...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [{'eventId': 6, 'sub..."
1,2565925,Atl\u00e9tico Madrid,679,Eibar,701,2,2,False,False,182.363636,...,0,0,2,0,"{'5': [], '10': [{'eventId': 10, 'subEventName...","{'5': [], '10': [{'eventId': 3, 'subEventName'...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [{'eventId': 2, 'subEventName'...","{'5': [{'eventId': 6, 'subEventName': '', 'tag..."
2,2565919,Athletic Club,678,Espanyol,691,0,1,False,True,182.818182,...,0,0,1,5,"{'5': [], '10': [{'eventId': 10, 'subEventName...","{'5': [], '10': [{'eventId': 3, 'subEventName'...","{'5': [], '10': [], '15': [{'eventId': 3, 'sub...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [{'eventId': 6, 'subEventName'..."
3,2565924,Valencia,674,Deportivo La Coru\u00f1a,677,2,1,True,False,181.909091,...,0,0,0,0,"{'5': [], '10': [{'eventId': 10, 'subEventName...","{'5': [], '10': [{'eventId': 3, 'subEventName'...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ..."
4,2565927,Villarreal,682,Real Madrid,675,2,2,False,False,182.454545,...,0,0,3,3,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...","{'5': [], '10': [{'eventId': 3, 'subEventName'...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2565552,Sevilla,680,Espanyol,691,1,1,False,False,182.181818,...,0,0,4,0,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [{'eventId...","{'5': [], '10': [], '15': [], '20': [], '25': ..."
376,2565551,Girona,756,Atl\u00e9tico Madrid,679,2,2,False,False,181.727273,...,0,1,4,2,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...","{'5': [], '10': [{'eventId': 3, 'subEventName'...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [{'eventId': 6, 'subEventName': '', 'tag..."
377,2565549,Celta de Vigo,692,Real Sociedad,687,2,3,False,True,179.545455,...,1,0,1,1,"{'5': [{'eventId': 10, 'subEventName': 'Shot',...","{'5': [], '10': [], '15': [{'eventId': 3, 'sub...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [{'eventId': 6, 'subEventName': '', 'tag..."
378,2565556,Valencia,674,Las Palmas,714,1,0,True,False,182.000000,...,1,1,3,1,"{'5': [], '10': [], '15': [{'eventId': 10, 'su...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [], '15': [], '20': [], '25': ...","{'5': [], '10': [{'eventId': 2, 'subEventName'...","{'5': [], '10': [{'eventId': 6, 'subEventName'..."


In [95]:
df_matches.to_csv(f'data/final_dfs/{country}.csv', index=False)